In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.animation as animation

from scipy.spatial.distance import cdist

In [73]:
# Setting parameters
# basic view
view_width = 1920
view_height = 1080

# anchor point
anchor_point = np.array([500, 700])


# window
window_width = 200
window_height = 200

init_pos = anchor_point.copy()
init_vel = np.array([0, 0])

max_v = 10

cur_pos = init_pos.copy()
cur_vel = init_vel.copy()

# obstacles
init_mask = np.zeros((view_width, view_height), dtype=bool)

# force and potential field
k_att = zeta =  10.0  # 吸引力系数
k_rep = eta = 10.0  # 排斥力系数

d0 = 200    # 障碍物影响范围

In [3]:
# 建立一个稍有重叠但是虚假的障碍物
obstacle_mask = np.zeros((view_width, view_height), dtype=bool)
obstacle_mask[500:600, 600:700] = True

obstacles = obstacle_mask

In [ ]:
print(obstacle_mask)
print(obstacles)

## Force

### attr

checked.

In [5]:
def get_attractive_force(position: np.ndarray, anchor: np.ndarray) -> np.ndarray:
    """
    计算对锚点的吸引力（单位向量）
    
    Args:
        position (np.ndarray): 当前位置
        anchor (np.ndarray): 锚点位置
    
    Returns:
        np.ndarray: 吸引力单位向量
    """
    attractive_force = anchor - position
    distance = np.linalg.norm(attractive_force)
    
    if distance > 0:
        # 只返回单位向量，不应用系数
        attractive_force = attractive_force / distance
    else:
        attractive_force = np.zeros(2)
    
    return attractive_force

In [ ]:
cur_pos[0] -= 10
cur_pos[1] -= 5

attractive_force = get_attractive_force(cur_pos, anchor_point)
attractive_force *= k_att
print(attractive_force)

### rep

In [ ]:
# 有以下两点计算公式
# rep_force = (1 / D - 1 / self.d_0) * (1 / D) ** 2 * (cur_pos - obstacles)
# 针对新的矩阵环境进行更新
# 0703 更新：基本验证没有问题， 接下来封装为函数，继续写更新部份

x_coords, y_coords  = np.mgrid[0:view_width, 0:view_height]
# 这里计算距离矩阵，都是欧式距离，没有负数，稍后用与排斥力系数计算
window_distances = np.sqrt((x_coords - cur_pos[0])**2 + (y_coords - cur_pos[1])**2)
anchor_distances = np.sqrt((x_coords - anchor_point[0])**2 + (y_coords - anchor_point[1])**2)

# 避免除零错误，设置最小距离
window_distances = np.maximum(window_distances, 1e-6)
anchor_distances = np.maximum(anchor_distances, 1e-6)

# 得到影响范围的mask
window_influence_mask = window_distances <= d0
anchor_influence_mask = anchor_distances <= d0
# 按位或进行合并
influence_mask = window_influence_mask | anchor_influence_mask
# 按位与进行筛选真正起作用的obstacles
final_mask = obstacle_mask & influence_mask

# 计算排斥力系数
# repulsive_coefficient = (1/D - 1/d0) * (1/D)^2
inv_distances = 1.0 / window_distances
inv_d0 = 1.0 / d0
repulsive_coefficient = (inv_distances - inv_d0) * (inv_distances ** 2)

# 只在有效障碍物位置应用排斥力；理论上这一部分应该放在力的计算完成之后，但这里也许能够降低一些运算量
repulsive_coefficient = np.where(final_mask, repulsive_coefficient, 0)

# 计算力的方向（从障碍物指向机器人）
# 这里减完有1单位的误差？索引从0开始
window_distances_x = cur_pos[0] - x_coords
window_distances_y = cur_pos[1] - y_coords

# 替换0值，避免除零
window_distances_x[window_distances_x == 0]= 1e-6
window_distances_y[window_distances_y == 0]= 1e-6

force_x = repulsive_coefficient * window_distances_x
force_y = repulsive_coefficient * window_distances_y

total_force_x = np.sum(force_x)
total_force_y = np.sum(force_y)

repulsive_force = np.array([total_force_x, total_force_y])
repulsive_force

## Visualization

In [ ]:
fig, ax = plt.subplots(figsize=(10, 8))

ax.set_xlim(-100, view_width + 100)
ax.set_ylim(-100, view_height + 100)

ax.set_title('Dynamic Artificial Potential Field')

# 绘制静态元素
anchor_plot, = ax.plot(anchor_point[0], anchor_point[1], 'go', markersize=10, label='Anchor Point')
# obstacles_plot, = ax.plot(obstacles[:, 0], obstacles[:, 1], 'ro', markersize=8, label='Obstacles')

window_plot, = ax.plot(cur_pos[0], cur_pos[1], 'bo', markersize=8, label='Window Center')

# 绘制吸引力箭头测试
quiver_robot = plt.arrow(cur_pos[0], cur_pos[1], 10*attractive_force[0], 10*attractive_force[1],
                         width=10, color='blue')

In [ ]:
# 外围图例
ax.legend(loc='upper left')
ax.set_xlabel('X Position')
ax.set_ylabel('Y Position')

plt.tight_layout()
plt.show()

# Toy Setting

方便检验和可视化确认

In [2]:
# 临时变小检验值
view_width = 20
view_height = 10
d0 = 3

cur_pos = np.array([10, 5])
anchor_point = np.array([5, 8])

In [3]:
# 设置obstacle mask
obstacle_mask = np.zeros((view_width, view_height), dtype=bool)
obstacle_mask[6:8, 6:8] = True

df = pd.DataFrame(obstacle_mask)
df

,0,1,2,3,4,5,6,7,8,9
0,False,False,False,False,False,False,False,False,False,False
1,False,False,False,False,False,False,False,False,False,False
2,False,False,False,False,False,False,False,False,False,False
3,False,False,False,False,False,False,False,False,False,False
4,False,False,False,False,False,False,False,False,False,False
5,False,False,False,False,False,False,False,False,False,False
6,False,False,False,False,False,False,True,True,False,False
7,False,False,False,False,False,False,True,True,False,False
8,False,False,False,False,False,False,False,False,False,False
9,False,False,False,False,False,False,False,False,False,False
